<a href="https://colab.research.google.com/github/aelhababi/py_rep_one/blob/master/test1_lr_tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np


In [0]:
import os

In [0]:
os.chdir("..")

In [0]:
os.getcwd()

In [0]:
import tensorflow as tf

In [0]:
x = tf.constant([5.2])

In [0]:
y = tf.Variable([5])

In [0]:
z = tf.constant([12.4])

In [0]:
y = y.assign([7])

In [0]:
with tf.Session() as sess:
  initialization = tf.global_variables_initializer()
  print(y.eval())

In [0]:
with tf.Session() as sess:
  initialization = tf.global_variables_initializer()
  sum1 = tf.add(x,z)
  print(sum1.eval())

In [0]:
zed = tf.constant([4])
new_sum = tf.add(sum1,z)

In [0]:
with tf.Session() as sess:
  print(new_sum.eval())

Chargement des données

In [0]:
import pandas as pd
import matplotlib.pyplot as plt

In [0]:
housing_dataframe = pd.read_csv("https://download.mlcc.google.com/mledu-datasets/california_housing_train.csv",sep=",")

In [0]:
housing_dataframe.head()

In [0]:
housing_dataframe = housing_dataframe.reindex(np.random.permutation(housing_dataframe.index))

In [0]:
housing_dataframe['median_house_value'] /= 1000.0

In [0]:
housing_dataframe.describe()

###Construction du modele


**Step 1**

In [0]:
#define the input feature
main_feature = housing_dataframe['total_rooms']
feature_column = tf.feature_column.numeric_column("total_rooms")

**Step 2 : definir la cible**

In [0]:
my_target = housing_dataframe["median_house_value"]

**configurer  la classe regression linéaire LinearRegressor**

In [0]:
#training the model


my_optimizer = tf.train.GradientDescentOptimizer(learning_rate = 0.0000001)

In [0]:
my_optimizer = tf.contrib.estimator.clip_gradients_by_norm(my_optimizer, 5.0)

In [0]:
#Configure LinearRegressor
lr = tf.estimator.LinearRegressor(feature_columns = feature_column,
                                 optimizer = my_optimizer)

**Step 4 Fonctions d'entrée**

In [0]:
from __future__ import print_function

import math

from IPython import display
from matplotlib import cm
from matplotlib import gridspec
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import metrics
import tensorflow as tf
from tensorflow.python.data import Dataset

tf.logging.set_verbosity(tf.logging.ERROR)
pd.options.display.max_rows = 10
pd.options.display.float_format = '{:.1f}'.format


In [0]:
california_housing_dataframe = pd.read_csv("https://download.mlcc.google.com/mledu-datasets/california_housing_train.csv", sep=",")

california_housing_dataframe = california_housing_dataframe.reindex(
    np.random.permutation(california_housing_dataframe.index))
california_housing_dataframe["median_house_value"] /= 1000.0
california_housing_dataframe

In [0]:
california_housing_dataframe.describe()

In [0]:
# Define the input feature: total_rooms.
my_feature = california_housing_dataframe[["total_rooms"]]

# Configure a numeric feature column for total_rooms.
feature_columns = [tf.feature_column.numeric_column("total_rooms")]

In [0]:
# Define the label.
targets = california_housing_dataframe["median_house_value"]

In [0]:
# Use gradient descent as the optimizer for training the model.
my_optimizer=tf.train.GradientDescentOptimizer(learning_rate=0.0000001)
my_optimizer = tf.contrib.estimator.clip_gradients_by_norm(my_optimizer, 5.0)

# Configure the linear regression model with our feature columns and optimizer.
# Set a learning rate of 0.0000001 for Gradient Descent.
linear_regressor = tf.estimator.LinearRegressor(
    feature_columns=feature_columns,
    optimizer=my_optimizer
)

In [0]:
def my_input_fn(features, targets, batch_size=1, shuffle=True, num_epochs=None):
    """Trains a linear regression model of one feature.
  
    Args:
      features: pandas DataFrame of features
      targets: pandas DataFrame of targets
      batch_size: Size of batches to be passed to the model
      shuffle: True or False. Whether to shuffle the data.
      num_epochs: Number of epochs for which data should be repeated. None = repeat indefinitely
    Returns:
      Tuple of (features, labels) for next data batch
    """
  
    # Convert pandas data into a dict of np arrays.
    features = {key:np.array(value) for key,value in dict(features).items()}                                           
 
    # Construct a dataset, and configure batching/repeating.
    ds = Dataset.from_tensor_slices((features,targets)) # warning: 2GB limit
    ds = ds.batch(batch_size).repeat(num_epochs)
    
    # Shuffle the data, if specified.
    if shuffle:
      ds = ds.shuffle(buffer_size=10000)
    
    # Return the next batch of data.
    features, labels = ds.make_one_shot_iterator().get_next()
    return features, labels

In [0]:
_ = linear_regressor.train(
    input_fn = lambda:my_input_fn(my_feature, targets),
    steps=100
)